In [1]:
import numpy as np
import pandas as pd

from pathlib import Path


### Plan:

1. transform data
1. use a logisitc regression model as a a baseline for prediciton of the trainng set
1. use a mulit-layed layered neural network to predict a binary prediction fo survival for he training set

#### compeition docs 
https://www.kaggle.com/competitions/titanic/data

#### data
data dictionary
        Variable	Definition	Key
        survival	- Survival	0 = No, 1 = Yes
        pclass - 	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
        sex	- Sex	
        Age	- Age in years	
        sibsp	- # of siblings / spouses aboard the Titanic	
        parch	- # of parents / children aboard the Titanic	
        ticket	- Ticket number	
        fare	- Passenger fare	
        cabin	- Cabin number	
        embarked	- Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

#### activation funciton:
- layer 1 - Relu
- output layer - sigmoid

#### formulas:
- logistic regression:
        sigmoid(w.T x  + b)
- sigmoid:
        np.sigmoid()
- relu:
        np.maximum(arr, 0)
- back propagation:

In [147]:
# load data

DATA_PATH = Path('/Users/tommadden/titanic_survival/data')
p = Path(DATA_PATH, 'titanic', 'train.csv')
df = pd.read_csv(p, index_col=0)
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [148]:
# transform input parameters
input_df = df

# sex classification: male = 1, female = 0
input_df['Sex'] = df.Sex.eq('male').mul(1)

# fill NaN age to average age (29.5)
input_df['Age'] = input_df['Age'].fillna(29.5)

input_df['Cabin']
input_df.Cabin.fillna(0, inplace = True)
# set records with multiple cabin assignments to first cabin assingment (assumes this will not introduce bias)
input_df['CabinLevel'] = [i[0] if type(i) == list else i for i in input_df.Cabin.str.split().values]
# regex to isolate first letter of cabin name (presumed to be the level of the ship)
input_df['CabinLevel'] = [i[0] if type(i) == list else i for i in input_df.Cabin.str.findall(r'[A-Z]').values]

# cabin counts to columns
input_df['CabinLevel'].fillna(0, inplace = True)
levels = input_df['CabinLevel'].unique()
for i in levels:
    input_df[str('CabinLevel'+str(i))] = np.where(input_df['CabinLevel'] == i, 1, 0)

    
    
    
# parse and store cabin number from cabin cloumn into canibno cloumn
input_df['CabinNo'] = [i[0] if type(i) == list else i for i in input_df.Cabin.str.split().values]

num_arr = input_df.CabinNo.str.findall(r'[0-9]+').values

CabinNoArr = []
for i in num_arr:
    if type(i) == list:
        try:
          CabinNoArr.append(i[0])  
        except:
            CabinNoArr.append(i)
    else:
        CabinNoArr.append(i)

input_df['CabinNo'] = CabinNoArr    
input_df['CabinNo'].fillna(0, inplace=True)

# include only unwanted comlumns
# filt = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
#        'Fare', 'Cabin', 'CabinLevel', 'CabinLevel0', 'CabinLevelC',
#        'CabinLevelE', 'CabinLevelG', 'CabinLevelD', 'CabinLevelA',
#        'CabinLevelB', 'CabinLevelF', 'CabinLevelT']
# input_df = input_df[filt]

# exclude unwanted columns
input_df = input_df.loc[:, ~input_df.columns.isin(['Survived','Name','Ticket', 'Embarked','Cabin','CabinLevel'])]


print(input_df.columns)
input_df

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'CabinLevel0',
       'CabinLevelC', 'CabinLevelE', 'CabinLevelG', 'CabinLevelD',
       'CabinLevelA', 'CabinLevelB', 'CabinLevelF', 'CabinLevelT', 'CabinNo'],
      dtype='object')


,Pclass,Sex,Age,SibSp,Parch,Fare,CabinLevel0,CabinLevelC,CabinLevelE,CabinLevelG,CabinLevelD,CabinLevelA,CabinLevelB,CabinLevelF,CabinLevelT,CabinNo
PassengerId,,,,,,,,,,,,,,,,
1,3,1,22.0,1,0,7.2500,1,0,0,0,0,0,0,0,0,0
2,1,0,38.0,1,0,71.2833,0,1,0,0,0,0,0,0,0,85
3,3,0,26.0,0,0,7.9250,1,0,0,0,0,0,0,0,0,0
4,1,0,35.0,1,0,53.1000,0,1,0,0,0,0,0,0,0,123
5,3,1,35.0,0,0,8.0500,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,2,1,27.0,0,0,13.0000,1,0,0,0,0,0,0,0,0,0
888,1,0,19.0,0,0,30.0000,0,0,0,0,0,0,1,0,0,42
889,3,0,29.5,1,2,23.4500,1,0,0,0,0,0,0,0,0,0


In [176]:
# pandas df to matrix & transpose so that there are 16 features and 891 training examples
# each column is a single training example, each column is a feature
input_matrix = input_df.values.T

assert input_matrix.shape == (16,891) 

input_matrix

array([[3, 1, 3, ..., 3, 1, 3],
       [1, 0, 0, ..., 0, 1, 1],
       [22.0, 38.0, 26.0, ..., 29.5, 26.0, 32.0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, '85', 0, ..., 0, '148', 0]], dtype=object)

In [6]:
# notes
arr = np.array([i * 0.01 for i in range(-10,10)])
# tanh fucntion
np.tanh(arr)

# relu function
np.maximum(arr,0)

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])

In [7]:
# deep learning specialisatoin logistic regression helper functions

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """
    s = 1 / (1+np.exp(1)**-z)
    return s


def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias) of type float
    """
    
    w = np.zeros(dim).reshape(dim,1)
    b = 0.0

    return w, b


def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]

    # forward propagation
    A = sigmoid(np.dot(w.T, X) + b)
    cost = - np.mean(Y * np.log(A) + (1 - Y ) * np.log(1 - A))
    
    # back propagation
    dw = 1/m * np.dot(X, (A-Y).T)
    db = np.mean(A-Y)
        
    cost = np.squeeze(np.array(cost))
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost


# GRADED FUNCTION: optimize

def optimize(w, b, X, Y, num_iterations=100, learning_rate=0.009, print_cost=False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    w = copy.deepcopy(w)
    b = copy.deepcopy(b)
    
    costs = []
    
    for i in range(num_iterations):
        # Cost and gradient calculation 
        grads, cost = propagate(w, b, X, Y)
                
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule 
        w = w - learning_rate * dw
        b = b - learning_rate * db

        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
            # Print the cost every 100 training iterations
            if print_cost:
                print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs


def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of yhat = y
    
    A = sigmoid(np.dot(w.T, X) + b) 
        
    for i in range(A.shape[1]):
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if A[0,i] >= 0.5 :
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
                    
    return Y_prediction



def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.5, print_cost=False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to True to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    # initialize parameters with zeros 
    w, b = initialize_with_zeros(X_train.shape[0])

    # calculates loss vs training set, calculates derivates and updates w & b
    params, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # updates w,b to optisied values
    w = params['w']
    b = params['b']
    
    # Predict test/train set examples
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
 
    # Print train/test Errors
    if print_cost:
        print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
        print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d